# LLM Event Extraction Demo

This notebook demonstrates how to use an LLM (OpenAI's `gpt-4o`) to extract calendar event details from text. Run each cell step-by-step to see the process.

**Presented:** March 08, 2025

In [ ]:
# Setup: Import libraries and initialize client
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import os

# Load API key from Codespaces Secrets or environment
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Set OPENAI_API_KEY in Codespaces Secrets or terminal (see 1Password).")
client = OpenAI(api_key=api_key)
model = "gpt-4o"
print("Client initialized successfully!")

In [ ]:
# Step 1: Define data models
class EventExtraction(BaseModel):
    description: str
    is_calendar_event: bool
    confidence_score: float

class EventDetails(BaseModel):
    name: str
    date: str  # ISO 8601
    duration_minutes: int
    participants: list[str]

class EventConfirmation(BaseModel):
    confirmation_message: str
    calendar_link: Optional[str] = None

print("Data models defined.")

In [ ]:
# Step 2: Extract event info
def extract_event_info(user_input: str) -> EventExtraction:
    today = datetime.now().strftime("%A, %B %d, %Y")
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": f"Today is {today}. Is this a calendar event?"},
            {"role": "user", "content": user_input},
        ],
        response_format=EventExtraction,
    )
    return completion.choices[0].message.parsed

# Test it
input_text = "Let's schedule a 1h team meeting next Tuesday at 2pm with Alice and Bob."
result = extract_event_info(input_text)
result

In [ ]:
# Step 3: Parse event details
def parse_event_details(description: str) -> EventDetails:
    today = datetime.now().strftime("%A, %B %d, %Y")
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": f"Today is {today}. Extract event details."},
            {"role": "user", "content": description},
        ],
        response_format=EventDetails,
    )
    return completion.choices[0].message.parsed

# Use previous output
details = parse_event_details(result.description)
details

In [ ]:
# Step 4: Generate confirmation
def generate_confirmation(event_details: EventDetails) -> EventConfirmation:
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": "Generate a confirmation. Sign off with 'Susie'."},
            {"role": "user", "content": str(event_details.model_dump())},
        ],
        response_format=EventConfirmation,
    )
    return completion.choices[0].message.parsed

# Use previous output
confirmation = generate_confirmation(details)
confirmation

In [ ]:
# Step 5: Full workflow
def process_calendar_request(user_input: str) -> Optional[EventConfirmation]:
    extraction = extract_event_info(user_input)
    if not extraction.is_calendar_event or extraction.confidence_score < 0.7:
        print("Not a calendar event or low confidence.")
        return None
    details = parse_event_details(extraction.description)
    return generate_confirmation(details)

# Run it all
result = process_calendar_request(input_text)
result

## Try Your Own!
Edit the `input_text` variable in Step 2 and rerun the cells below it to see how it works with your input.